<a href="https://colab.research.google.com/github/ResByte/llm-notebooks/blob/main/notebooks/02_RAG_with_local_llms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Local LLMs
This notebook runs local LLMs using `llama-cpp-python`

Installations

In [1]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install -q llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


## Model Setup

In [2]:
!pip install -q llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.

In [3]:
model_url = "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf?download=true"

In [4]:
from llama_index.llms import LlamaCPP

In [5]:
from llama_index.llms.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

In [6]:
llm  = LlamaCPP(
    model_url=model_url,
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    context_window=3900,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers":1},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True
)

total size (MB): 4368.44


4167it [00:20, 198.81it/s]                          
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [7]:
response = llm.complete('Tell me about Tokyo?')
print(response.text)

 Tokyo is the capital and largest city of Japan, located on the eastern coast of Honshu, the main island of Japan. It is a bustling metropolis with a population of over 13 million people. Tokyo is known for its modern skyscrapers, busy streets, and vibrant culture. The city has a rich history dating back to the Edo period (1603-1868) when it was the center of political power in Japan. Today, Tokyo is a major global financial center and a hub for business, technology, and innovation. Some popular attractions in Tokyo include the Tokyo Tower, the Imperial Palace, and the Tsukiji Fish Market.


## Query Engine

In [8]:
from llama_index import set_global_tokenizer
from transformers import AutoTokenizer

In [9]:
set_global_tokenizer(
    AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf").encode
)

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [10]:
# using hf embeddings
from llama_index.embeddings import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [11]:
from llama_index import ServiceContext

In [12]:
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model
)

## Dataset Setup

In [13]:
!mkdir -p 'data/paul_graham/'
!curl 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -o 'data/paul_graham/paul_graham_essay.txt'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 75042  100 75042    0     0   203k      0 --:--:-- --:--:-- --:--:--  204k


In [14]:
from llama_index import SimpleDirectoryReader

In [15]:
documents = SimpleDirectoryReader(
    "./data/paul_graham"
).load_data()

In [18]:
from llama_index import VectorStoreIndex

In [19]:
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)

In [20]:
query_engine = index.as_query_engine()

In [21]:
response = query_engine.query("What did the author do growing up?")
print(response)

Llama.generate: prefix-match hit


 The author grew up writing short stories and programming on an IBM 1401 computer in school. They didn't have any data stored on punched cards to input into programs, so they couldn't do much with the computer. With microcomputers, they started programming in earnest and wrote simple games, a program to predict how high their model rockets would fly, and a word processor. They also studied philosophy in college but switched to AI after finding it boring.


In [22]:
response.source_nodes[0].get_text()

'What I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.\n\nThe language we used was an early version of Fortran. You had to type programs on punch cards, then stack

In [23]:
response.source_nodes[1].get_text()

"I certainly did. So at the end of the summer Dan and I switched to working on this new dialect of Lisp, which I called Arc, in a house I bought in Cambridge.\n\nThe following spring, lightning struck. I was invited to give a talk at a Lisp conference, so I gave one about how we'd used Lisp at Viaweb. Afterward I put a postscript file of this talk online, on paulgraham.com, which I'd created years before using Viaweb but had never used for anything. In one day it got 30,000 page views. What on earth had happened? The referring urls showed that someone had posted it on Slashdot. [10]\n\nWow, I thought, there's an audience. If I write something and put it on the web, anyone can read it. That may seem obvious now, but it was surprising then. In the print era there was a narrow channel to readers, guarded by fierce monsters known as editors. The only way to get an audience for anything you wrote was to get it published as a book, or in a newspaper or magazine. Now anyone could publish anyt

## Evaluate

In [26]:
from llama_index.evaluation import generate_question_context_pairs
from llama_index.node_parser import SimpleNodeParser
from llama_index import VectorStoreIndex

In [30]:
# Build index with a chunk_size of 512
node_parser = SimpleNodeParser.from_defaults(chunk_size=512)
nodes = node_parser.get_nodes_from_documents(documents)
vector_index = VectorStoreIndex(nodes, service_context=service_context)

In [31]:
qa_dataset = generate_question_context_pairs(
    nodes,
    llm=llm,
    num_questions_per_chunk=2
)

100%|██████████| 69/69 [47:01<00:00, 40.89s/it]


In [32]:
from llama_index.evaluation import RetrieverEvaluator

In [33]:
retriever = vector_index.as_retriever(similarity_top_k=2)

In [34]:
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=retriever
)

In [42]:
import nest_asyncio
nest_asyncio.apply()

In [43]:
eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset)

In [44]:
eval_results

[RetrievalEvalResult(query='What programming language did you use on the IBM 1401 and what was it used for?', expected_ids=['b9e7521f-3ecd-4e6f-98f9-edb4514df3da'], expected_texts=None, retrieved_ids=['812ba9c2-5960-4fc7-99cc-29e5b48f5fb7', '16a1f619-13c2-4743-beec-52d04df0d01f'], retrieved_texts=["The result would ordinarily be to print something on the spectacularly loud printer.\n\nI was puzzled by the 1401. I couldn't figure out what to do with it. And in retrospect there's not much I could have done with it. The only form of input to programs was data stored on punched cards, and I didn't have any data stored on punched cards. The only other option was to do things that didn't rely on any input, like calculate approximations of pi, but I didn't know enough math to do anything interesting of that type. So I'm not surprised I can't remember any programs I wrote, because they can't have done much. My clearest memory is of the moment I learned it was possible for programs not to termi

In [46]:
import pandas as pd


def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()

    metric_df = pd.DataFrame(
        {"Retriever Name": [name], "Hit Rate": [hit_rate], "MRR": [mrr]}
    )

    return metric_df

In [47]:
display_results("top-2 eval", eval_results)

,Retriever Name,Hit Rate,MRR
0,top-2 eval,0.717391,0.634058
